In [1]:
import numpy as np
import pandas as pd
import psycopg2
import sys, os
import pandas.io.sql as psql

## Variabili postgres e dichiarazione della funzione di lettura

In [2]:
PGHOST="localhost"
PGDATABASE="extruder1407"
PGUSER="postgres"
PGPASSWORD="A"

In [3]:
def load_data(schema, table):
    conn_string = "host="+ PGHOST +" port="+ "5432" +" dbname="+ PGDATABASE +" user=" + PGUSER \
    +" password="+ PGPASSWORD
    conn=psycopg2.connect(conn_string)
    print("Connected!")
    cursor = conn.cursor()
    sql_command = "SELECT * FROM {}.{};".format(str(schema), str(table))
    #sql_command = "SELECT * FROM {}.{} ORDER BY id ASC LIMIT 200;".format(str(schema), str(table))
    print (sql_command)

    # Load the data
    data = pd.read_sql(sql_command, conn)

    print(data.shape)
    cursor.close()
    conn.close()
    return (data)

## Leggo tabella "tb_timeseriestag" ed elimino le colonne superflue

In [14]:
df_tag=load_data("public","tb_timeseriestag")
df_tag.drop(columns=['unit', 'datatype', 'description'], inplace=True)

Connected!
SELECT * FROM public.tb_timeseriestag;
(116, 5)


## Leggo tabella "tb_timeseriesdata" ed elimino le colonne superflue

In [6]:
df_timeseries=load_data("public","tb_timeseriesdata")

Connected!
SELECT * FROM public.tb_timeseriesdata;
(2795736, 119)


In [8]:
df_timeseries.drop(columns=['id', 'timestamplocale'], inplace=True)

In [9]:
df_timeseries['utctimestamp'] = df_timeseries['utctimestamp'].astype('datetime64[s]')

## Memorizzo il dataset in formato csv

In [18]:
df_timeseries.to_csv('files/extruder1407.csv', index=False)

In [16]:
df_tag.to_csv('files/tag_extruder1407.csv', index=False)

## Mappo le colonne con i nomi parlanti contenuti in timeseriestag

In [24]:
df_timeseries.columns = df_timeseries.columns.str.strip('ch_')

In [26]:
df_timeseries.set_index('utctimestamp', inplace=True)

In [27]:
dict_tag_extruder=dict(zip(df_tag["id"].astype(str), df_tag["tagname"]))
df_timeseries.columns = df_timeseries.columns.map(dict_tag_extruder) 
df_timeseries

,A-MeltPress-Die,A-MonomerFlap,A-QuenchingCab-Press,A-CoolingAir-FlowRate,A2-CoolingAir-FlowRate,A-ProcessAir-Ratio,A-Extruder-Speed,A-SpinPump-Speed,A-SuctionAirFan-Speed,A2-SuctionAirFan-Speed,...,L-Dry-exitRoll-relat,L-Dry-exitRoll-speed-ACT,L-Dry-intkRoll-curr-ACT,L-Dry-intkRoll-relat,L-Dry-intkRoll-speed-ACT,L-Dry-outRoll-curr-ACT,L-Dry-outRoll-relat,L-Dry-outRoll-speed-ACT,L-Dry-temp-ACT,L-Dry-temp-output
utctimestamp,,,,,,,,,,,,,,,,,,,,,
2021-06-02 15:33:52,48.000000,3752.0,-120.000000,0.0,0.000000,-1.0,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-02 15:33:53,48.000000,3752.0,-120.000000,0.0,0.000000,-1.0,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-02 15:33:54,48.000000,3752.0,-120.000000,0.0,0.000000,-1.0,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-02 15:33:56,48.000000,3752.0,-120.000000,0.0,0.000000,-1.0,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-02 15:33:57,48.000000,3752.0,-120.000000,0.0,0.000000,-1.0,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-14 07:06:39,72.164391,5440.0,2893.518555,5829.0,21268.228516,79.0,64.599609,55.998230,70.012184,95.012787,...,27.293478,640.899414,20.220192,0.0,639.558044,-5.412505,0.0,639.636902,28.700003,0.0
2021-07-14 07:06:40,72.164391,5440.0,2898.148193,5829.0,21268.228516,79.0,64.736328,56.021118,69.999001,94.987198,...,27.293478,640.856628,20.185446,0.0,639.536926,-5.440485,0.0,639.656250,28.700003,0.0
2021-07-14 07:06:41,72.280128,5440.0,2898.148193,5817.0,21297.742187,79.0,64.471436,56.007385,69.959465,94.999992,...,27.293478,640.937622,20.204998,0.0,639.593750,-5.460928,0.0,639.673889,28.700003,0.0


## Memorizzo il dataset con i nomi aggiornati in formato csv

In [30]:
df_timeseries.to_csv('files/extruder1407_tagged.csv', index=True)

In [49]:
df_timeseries=pd.read_csv('files/extruder1407_tagged.csv')  

In [33]:
df_timeseries

,utctimestamp,A-MeltPress-Die,A-MonomerFlap,A-QuenchingCab-Press,A-CoolingAir-FlowRate,A2-CoolingAir-FlowRate,A-ProcessAir-Ratio,A-Extruder-Speed,A-SpinPump-Speed,A-SuctionAirFan-Speed,...,L-Dry-exitRoll-relat,L-Dry-exitRoll-speed-ACT,L-Dry-intkRoll-curr-ACT,L-Dry-intkRoll-relat,L-Dry-intkRoll-speed-ACT,L-Dry-outRoll-curr-ACT,L-Dry-outRoll-relat,L-Dry-outRoll-speed-ACT,L-Dry-temp-ACT,L-Dry-temp-output
0,2021-06-02 15:33:52,48.000000,3752.0,-120.000000,0.0,0.000000,-1.0,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-06-02 15:33:53,48.000000,3752.0,-120.000000,0.0,0.000000,-1.0,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-06-02 15:33:54,48.000000,3752.0,-120.000000,0.0,0.000000,-1.0,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-06-02 15:33:56,48.000000,3752.0,-120.000000,0.0,0.000000,-1.0,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-06-02 15:33:57,48.000000,3752.0,-120.000000,0.0,0.000000,-1.0,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795731,2021-07-14 07:06:39,72.164391,5440.0,2893.518555,5829.0,21268.228516,79.0,64.599609,55.998230,70.012184,...,27.293478,640.899414,20.220192,0.0,639.558044,-5.412505,0.0,639.636902,28.700003,0.0
2795732,2021-07-14 07:06:40,72.164391,5440.0,2898.148193,5829.0,21268.228516,79.0,64.736328,56.021118,69.999001,...,27.293478,640.856628,20.185446,0.0,639.536926,-5.440485,0.0,639.656250,28.700003,0.0
2795733,2021-07-14 07:06:41,72.280128,5440.0,2898.148193,5817.0,21297.742187,79.0,64.471436,56.007385,69.959465,...,27.293478,640.937622,20.204998,0.0,639.593750,-5.460928,0.0,639.673889,28.700003,0.0
2795734,2021-07-14 07:06:42,72.164391,5440.0,2902.777832,5817.0,21297.742187,79.0,64.445801,56.011963,69.999001,...,27.293478,640.928650,20.183586,0.0,639.607605,-5.453708,0.0,639.688599,28.700003,0.0


In [50]:
df_timeseries.set_index('utctimestamp', inplace=True)

In [36]:
df_timeseries.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2795736 entries, 2021-06-02 15:33:52 to 2021-07-14 07:06:44
Columns: 116 entries, A-MeltPress-Die to L-Dry-temp-output
dtypes: float64(116)
memory usage: 2.4+ GB


In [52]:
df_timeseries.index = pd.to_datetime(df_timeseries.index, unit='ns')

## Per consentire l'analisi dei dati, effettuo un resampling della frequenza delle TS a 20 secondi, memorizzando in formato csv il nuovo dataset

In [60]:
df_resampled=df_timeseries.resample('20s').mean()

In [61]:
df_resampled.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 179920 entries, 2021-06-02 15:33:40 to 2021-07-14 07:06:40
Freq: 20S
Columns: 116 entries, A-MeltPress-Die to L-Dry-temp-output
dtypes: float64(116)
memory usage: 160.6 MB


In [62]:
df_resampled.to_csv('files/extruder1407_tagged_20s.csv', index=True)